In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether_v2 as birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

def norm(x,df):
    lo = df.min()
    hi = df.max()
    return (x - lo) / (hi - lo +0.00000001)

def dominate(_df,t,row_project_name,goals):
    wins = 0
    for i in range(_df.shape[0]):
        project_name = _df.iloc[i].name
        row = _df.iloc[i].tolist()
        if project_name != row_project_name:
            if dominationCompare(row, t,goals,_df):
                wins += 1
    return wins

def dominationCompare(other_row, t,goals,df):
    n = len(goals)
    weight = {'recall':1,'precision':1,'pf':-1.5}
    sum1, sum2 = 0,0
    for i in range(len(goals)):
        _df = df[goals[i]]
        w = weight[goals[i]]
        x = t[i]
        y = other_row[i]
        x = norm(x,_df)
        y = norm(y,_df)
        sum1 = sum1 - math.e**(w * (x-y)/n)
        sum2 = sum2 - math.e**(w * (y-x)/n)
    return sum1/n < sum2/n

In [3]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,data_store,clusters,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth != None:
            cluster_ids.append(key)
            if cluster_tree[key].depth not in cluster_structure.keys():
                cluster_structure[cluster_tree[key].depth] = {}
            cluster_structure[cluster_tree[key].depth][key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    goals = ['recall','precision','pf']
    for _ in range(1):
        score_df = pd.read_csv(data_source1 + '/bellwether_cdom_2.csv')
        score_df.drop(labels = ['Unnamed: 0'], axis = 1 ,inplace = True)
        level_1_bellwethers = {}
        for cluster in cluster_structure[2].keys():
            if cluster_structure[2][cluster] not in level_1_bellwethers.keys():
                level_1_bellwethers[cluster_structure[2][cluster]] = []
            level_1_bellwethers[cluster_structure[2][cluster]].append(score_df[score_df['id'] == cluster].bellwether.values[0])
        for cluster in cluster_structure[1].keys():
            if cluster not in level_1_bellwethers.keys():
                level_1_bellwethers[cluster] = []
            level_1_bellwethers[cluster] = list(df_train.iloc[cluster_tree[cluster].data_points].index)
        bell_df = {}
        for key in  level_1_bellwethers.keys():
            sub_cluster_bellwethers = level_1_bellwethers[key]
            final_score = pd.read_pickle(data_source1 + '/cdom_level1/cluster_'  + str(key) + '_performance.pkl')
            _results = {}
            for goal in goals:    
                for s_project in final_score.keys():
                    if s_project not in _results.keys():
                        _results[s_project] = {}
                        _temp = []
                    for d_projects in final_score[s_project].keys():
                        if goal == 'g':
                            _goal = 'g-score'
                        else:
                            _goal = goal
                        _score = np.median(final_score[s_project][d_projects][_goal])
                        _temp.append(np.median(final_score[s_project][d_projects][_goal]))
                    if goal not in _results[s_project].keys():
                        _results[s_project][goal] = []
                    _results[s_project][goal] = np.median(_temp)
            _df = pd.DataFrame.from_dict(_results, orient = 'index')
            dom_score = []
            for row_id in range(_df.shape[0]):
                project_name = _df.iloc[row_id].name
                row = _df.iloc[row_id].tolist()
                wins = dominate(_df,row,project_name,goals)
                dom_score.append(wins)
            _df['wins'] = dom_score
            c_dom = _df.wins.values.tolist()
            best_project = _df.index[c_dom.index(max(c_dom))]
            best_project_perf = _df.loc[best_project].values.tolist()
            best_project_perf.append(best_project)
            bell_df[key] = best_project_perf
        perf_df = pd.DataFrame.from_dict(bell_df, orient = 'index', columns = ['recall','precision','pf','cdom','bellwether'])    
        perf_df.to_csv(data_source1 + '/bellwether_cdom_1.csv')
            
            
            

In [5]:
for i in range(20):
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf/2/fold_' + str(i)
    data_store = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf/1/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    find_bellwether(data_source1,data_store,clusters,path,i)

{3: [0.21, 0.325, 0.25, 4.0, 'pentahoanalysistool.csv'], 9: [0.765, 0.48, 0.365, 9.0, 'sylpheed-claws.csv'], 20: [0.405, 0.32, 0.26, 15.0, 'gmerlin.csv'], 38: [0.555, 0.33, 0.22, 19.0, 'qtractor.csv'], 1: [1.0, 1.0, 1.0, 0.0, 'benojt.csv'], 2: [1.0, 1.0, 1.0, 0.0, 'emftriple.csv'], 37: [0.575, 0.94, 0.26, 1.0, 'ivef-sdk.csv']}
{2: [0.445, 0.42, 0.31, 19.0, 'dataobjectsdotnet.csv'], 23: [0.04, 0.48, 0.3, 6.0, 'foursquared.csv'], 31: [0.315, 0.515, 0.14500000000000002, 3.0, 'fluent-nhibernate.csv'], 36: [0.7949999999999999, 0.44, 0.26, 11.0, 'autofac.csv'], 49: [0.54, 0.23, 0.19, 13.0, 'sepgsql.csv'], 1: [0.86, 0.9299999999999999, 0.86, 0.0, 'atunes.csv']}
{1: [0.88, 0.38, 0.15, 8.0, 'ninject.csv'], 12: [0.5, 0.53, 0.03, 1.0, 'emftriple.csv'], 15: [0.32, 0.4, 0.29, 12.0, 'unimrcp.csv'], 29: [0.375, 0.33, 0.22, 19.0, 'ktc.csv'], 50: [0.43, 0.595, 0.43, 4.0, 'adaptagrams.csv'], 56: [0.27, 0.72, 0.28500000000000003, 2.0, 'vapor.csv'], 60: [0.32, 0.485, 0.24, 8.0, 'codepurge.csv'], 11: [1.0,